In [18]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
model_sentence = SentenceTransformer("paraphrase-MiniLM-L6-v2")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/onur/.pyenv/versions/3.11.2/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [100]:
import joblib 
joblib.dump(model_sentence, 'model_sentence.pkl')

['model_sentence.pkl']

In [4]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [5]:
df_original = pd.read_csv('clustered_data_v2.csv')

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
df_original.head()

,price,bedrooms,bathrooms,sqft_lot_normalized,floors,waterfront,view,condition,yr_built,yr_renovated,zipcode,lat,long,nearest_station_distance_km,bathroom_category,density,commute_time,distance_to_point_km,is_near_shore,grade_living_normalized,bedrooms_cat,lot_size_category,grade_living_category,outliers_ecod,cluster_all_data,cluster_no_outlier,sqft_lot,grade_living
0,221900.0,3,1.00,8.639588,1.0,0,0,3,1955,0,98178,47.5112,-122.257,8620.002287,Studio or 1 Bathroom,5364.0,32.9,0.595763,False,9.019301,3-4,0,0,0,1,1.0,5650,8260
1,538000.0,3,2.25,8.887791,2.0,0,0,3,1951,1991,98125,47.7210,-122.319,2078.181042,2-3 Bathrooms,7999.0,28.9,3.038596,False,9.797627,3-4,1,2,0,0,0.0,7242,17990
2,180000.0,2,1.00,9.210440,1.0,0,0,3,1933,0,98028,47.7379,-122.233,8109.552186,Studio or 1 Bathroom,3833.0,33.1,2.317669,False,8.438366,<3,2,0,0,1,1.0,10000,4620
3,604000.0,4,3.00,8.517393,1.0,0,0,5,1965,0,98136,47.5208,-122.393,8918.318639,2-3 Bathrooms,7517.0,30.6,0.101668,True,9.526683,3-4,0,1,0,2,0.0,5000,13720
4,510000.0,3,2.00,8.997271,1.0,0,0,3,1987,0,98074,47.6168,-122.045,19789.800401,1-2 Bathrooms,2896.0,32.7,1.750656,False,9.506065,3-4,1,1,0,0,2.0,8080,13440


In [122]:
cluster_columns = ['bathroom_category', 'lot_size_category', 'floors','view', 'condition', 'is_near_shore','grade_living_category']

# XGBOOST

In [123]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

X = df_original[cluster_columns]
y = df_original['cluster_all_data']

# rename X columns for < and > signs
X.columns = X.columns.str.replace('<', 'less_than')
X.columns = X.columns.str.replace('>', 'greater_than')

numerical_cols = ['floors','view', 'condition']
categorical_cols = ['bathroom_category', 'lot_size_category','grade_living_category']

# Assume X is your features and y is your target


# Assume numerical_cols is a list of numerical column names, and categorical_cols is a list of categorical column names
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

# Preprocessing
X = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [124]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report


# Assuming embeddings and labels are your data and target variables respectively
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an XGBoost model
model = xgb.XGBClassifier(objective='multi:softmax', eval_metric='logloss')  # Change objective to 'multi:softprob' for multi-class

# Train the model
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluation
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


Accuracy: 0.9070090215128384
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      3115
           1       1.00      1.00      1.00       781
           2       0.57      0.23      0.33       427

    accuracy                           0.91      4323
   macro avg       0.82      0.73      0.75      4323
weighted avg       0.89      0.91      0.89      4323



In [60]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

model = xgb.XGBClassifier(objective='multi:softmax', eval_metric='logloss') 

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print(f'Best parameters: {grid_search.best_params_}')
y_pred = grid_search.best_estimator_.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 243 candidates, totalling 1215 fits


Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Accuracy: 0.7957436965070552
              precision    recall  f1-score   support

           0       0.81      0.94      0.87      3115
           1       0.73      0.54      0.62       781
           2       0.63      0.22      0.33       427

    accuracy                           0.80      4323
   macro avg       0.72      0.57      0.61      4323
weighted avg       0.78      0.80      0.77      4323



# Neural Networks

In [74]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

X = df_original[cluster_columns]
y = df_original['cluster_all_data']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.columns = X_train.columns.str.replace(r'[<>]', lambda x: 'less_than' if x.group() == '<' else 'greater_than', regex=True)
X_test.columns = X_test.columns.str.replace(r'[<>]', lambda x: 'less_than' if x.group() == '<' else 'greater_than', regex=True)


numerical_cols = ['floors','view', 'condition']
categorical_cols = ['bedrooms_cat', 'lot_size_category','grade_living_category']
# Assume X is your features and y is your target

# Assume numerical_cols is a list of numerical column names, and categorical_cols is a list of categorical column names
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

# Preprocessing
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train.values.reshape(-1, 1)).toarray()
y_test = encoder.transform(y_test.values.reshape(-1, 1)).toarray()

# Convert one-hot encoded labels to 1D arrays
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)


print(X_train.shape, y_train.shape)



(17290, 12) (17290,)


In [84]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report


# Encode labels if they are categorical
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Build the model
model = Sequential()
model.add(Dense(128, input_dim=12, activation='relu'))  # Adjust input_dim to match the number of features
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Output layer for multiclass classification with 3 clas


# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # Use 'categorical_crossentropy' for multi-class

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Predict
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print(classification_report(y_test, y_pred))

Epoch 1/20


/Users/onur/.pyenv/versions/3.11.2/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


433/433 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step - accuracy: 0.7321 - loss: 0.6838 - val_accuracy: 0.7741 - val_loss: 0.5185
Epoch 2/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - accuracy: 0.7777 - loss: 0.5471 - val_accuracy: 0.7848 - val_loss: 0.5089
Epoch 3/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step - accuracy: 0.7879 - loss: 0.5218 - val_accuracy: 0.7828 - val_loss: 0.4994
Epoch 4/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step - accuracy: 0.7848 - loss: 0.5240 - val_accuracy: 0.7880 - val_loss: 0.4971
Epoch 5/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 644us/step - accuracy: 0.7785 - loss: 0.5212 - val_accuracy: 0.7796 - val_loss: 0.4966
Epoch 6/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 611us/step - accuracy: 0.7799 - loss: 0.5208 - val_accuracy: 0.7955 - val_loss: 0.4943
Epoch 7/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 610us/step - accuracy: 0.7816 - loss: 0.5177 - val_accuracy: 0.7961 - val_loss: 0.4935
Epoch 8/20
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step - accuracy: 0.7877 - loss: 0.5102 - val_accurac

# LLM + XGBOOST

In [ ]:
def compile_text(x):
    text =  f"""Bedrooms: {x['bedrooms_cat']}, 
                Lot Size (sqft): {x['lot_size_category']}, 
                Floors: {x['floors']}, 
                View: {x['view']}, 
                Condition: {x['condition']}, 
                Bathroom Category: {x['bathroom_category']}, 
                Near Shore: {x['is_near_shore']}, 
                Grade Living: {x['grade_living_category']}
            """
    return text

In [125]:
def compile_text(x):
    text =  f"""Bathroom Category: {x['bathroom_category']}, 
                Near Shore: {x['is_near_shore']}, 
            """
    return text

In [126]:
sentences = df_original.apply(lambda x: compile_text(x), axis=1).tolist()

In [127]:
output = model_sentence.encode(sentences=sentences, show_progress_bar= True, normalize_embeddings  = True)

Batches: 100%|██████████| 676/676 [00:14<00:00, 46.99it/s]


In [128]:
output[1].shape

(384,)

In [129]:
df_embedding = pd.DataFrame(output)

In [130]:
df_embedding

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383
0,0.001881,-0.047290,0.073722,0.011747,-0.027247,-0.016082,-0.058573,-0.029313,0.035192,0.044149,-0.002745,-0.102086,0.079029,0.046736,0.020035,-0.037034,0.045304,0.003064,0.005518,0.040733,0.048064,0.055127,-0.044484,0.020897,0.040543,-0.003770,-0.035720,0.044637,0.009856,-0.083843,-0.071919,-0.005554,-0.059270,-0.014939,0.065378,-0.027292,-0.052032,0.005903,0.012294,-0.033045,-0.041202,0.042874,-0.001613,0.038566,0.012130,0.063305,-0.043766,-0.000777,-0.012668,0.008737,0.054265,0.041289,-0.040580,0.100461,-0.067998,-0.044571,-0.099581,-0.015500,-0.081397,0.037531,-0.004924,0.049479,-0.087074,0.072380,-0.022246,0.095600,0.011049,0.051625,0.020084,0.003119,-0.035710,-0.007741,-0.083184,0.004589,0.043884,-0.039086,0.007067,0.058822,-0.043758,-0.022541,-0.041367,-0.054098,0.052061,0.045308,-0.030424,-0.030677,-0.041067,0.014950,-0.052664,-0.048059,-0.020601,-0.030324,0.011030,-0.047675,0.008113,-0.037856,-0.102813,0.027609,0.066042,0.004160,-0.046952,-0.019739,-0.016845,-0.038853,0.095002,-0.019060,0.030924,0.011776,-0.002012,0.048162,0.003633,-0.065250,0.008367,-0.068894,-0.018336,-0.027860,0.073091,-0.054308,-0.046734,0.001171,0.023461,-0.012699,0.008501,0.002351,-0.062857,-0.016105,-0.020568,0.019448,0.013467,-0.042975,0.029098,-0.024237,0.076241,0.048447,0.054392,0.069461,0.055439,0.003006,0.094572,0.004700,-0.074781,0.013517,0.008267,0.023986,-0.021836,0.057409,-0.083145,0.008059,-0.003399,0.067897,0.018654,0.085905,0.050988,-0.083571,-0.040075,-0.063140,-0.035657,0.068806,-0.036812,-0.015583,-0.003421,0.027547,-0.046234,-0.009961,-0.067576,0.005955,-0.058931,-0.090702,-0.003322,0.031617,0.058533,0.048530,0.121001,-0.008023,-0.028199,0.016124,-0.006789,0.009613,0.020967,0.085329,-0.083556,0.119142,0.005298,-0.041754,0.044433,0.026071,-0.017281,0.056710,-0.018535,0.119274,-0.039392,-0.068618,0.010662,0.021315,0.005878,0.053222,0.089602,-0.115902,-0.121481,-0.070433,-0.048345,0.074339,-0.006837,-0.018117,0.024900,-0.017777,0.012897,0.020066,-0.075821,0.052528,-0.016137,0.088185,-0.132549,-0.066378,0.040378,0.017280,-0.032639,-0.013031,0.018404,-0.004449,0.002405,-0.019861,0.011933,-0.056640,0.016069,-0.027150,-0.049485,0.088610,-0.010103,-0.000118,0.036867,-0.057338,0.000514,0.057461,-0.077601,0.086519,-0.005886,-0.012882,0.004315,0.054886,0.045777,-0.043640,-0.062697,0.178492,-0.027090,0.027352,0.016352,-0.033960,-0.008090,0.117862,-0.087594,0.040700,0.002433,0.048692,-0.103476,0.047403,-0.003210,-0.040199,0.026832,-0.044416,0.039274,0.017298,-0.089811,0.023662,0.092844,-0.062318,-0.062264,0.045853,-0.028575,-0.013734,-0.0452

In [131]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

X = df_embedding.copy()
y = df_original['cluster_all_data']

# Assuming embeddings and labels are your data and target variables respectively
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an XGBoost model
model = xgb.XGBClassifier(objective='multi:softmax', eval_metric='logloss')  # Change objective to 'multi:softprob' for multi-class

# Train the model
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluation
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


Accuracy: 0.9990747166319686
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3115
           1       0.99      1.00      1.00       781
           2       1.00      0.99      1.00       427

    accuracy                           1.00      4323
   macro avg       1.00      1.00      1.00      4323
weighted avg       1.00      1.00      1.00      4323



In [132]:
import joblib 
joblib.dump(model, 'model_classification_xgboost.pkl')


['classification_xgboost_model.pkl']

In [133]:
# import xgboost as xgb
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.metrics import accuracy_score, classification_report

# # Assuming embeddings and labels are your data and target variables respectively
# X = df_embedding.copy()
# y = df_original['cluster_all_data']

# # Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create an XGBoost model
# model = xgb.XGBClassifier(objective='multi:softmax', eval_metric='mlogloss', use_label_encoder=False)

# # Define the parameter grid for hyperparameter tuning
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [3, 5, 7],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'subsample': [0.8, 1.0],
#     'colsample_bytree': [0.8, 1.0]
# }

# # Create a GridSearchCV object
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)

# # Fit the model using grid search
# grid_search.fit(X_train, y_train)

# # Get the best model from grid search
# best_model = grid_search.best_estimator_

# # Print the best parameters
# print("Best parameters found: ", grid_search.best_params_)

# # Predict using the best model
# y_pred = best_model.predict(X_test)

# # Evaluation
# print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
# print(classification_report(y_test, y_pred))


# LLM + Neural Networks

In [134]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

X = df_embedding.copy()
y = df_original['cluster_all_data']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train.values.reshape(-1, 1)).toarray()
y_test = encoder.transform(y_test.values.reshape(-1, 1)).toarray()

# Convert one-hot encoded labels to 1D arrays
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

print(X_train.shape, y_train.shape)


(17290, 384) (17290,)


In [135]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report


# Encode labels if they are categorical
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Build the model
model = Sequential()
model.add(Dense(128, input_dim=384, activation='relu'))  # Adjust input_dim to match the number of features
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Output layer for multiclass classification with 3 clas


# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # Use 'categorical_crossentropy' for multi-class

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

# Predict
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print(classification_report(y_test, y_pred))

/Users/onur/.pyenv/versions/3.11.2/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


136/136 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step
Accuracy: 0.9990747166319686
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3115
           1       0.99      1.00      1.00       781
           2       1.00      0.99      1.00       427

    accuracy                           1.00      4323
   macro avg       1.00      1.00      1.00      4323
weighted avg       1.00      1.00      1.00      4323

